##### Classify

In [1]:
import pandas as pd
import json

# df = pd.read_excel('../data/Elsevier-CS/Texts_3000.xlsx')
df = pd.read_excel('../data/Elsevier-LIS/Texts.xlsx')
ats = df['Abstract'].tolist()
hts = df['Highlights'].tolist()
links = df['Pii'].tolist()

In [2]:
ht_sens, at_sens = [], []
ht_links, at_links = [], []

for i, at in enumerate(ats):
    at_sens.extend(at.split('.'))
    at_links.extend([links[i]]*len(at.split('.')))
    ht_sens.extend(hts[i].split(';'))
    ht_links.extend([links[i]]*len(hts[i].split(';')))

print("The number of sentence  highlight:{}  abstract:{}".format(len(ht_sens), len(at_sens)))

The number of sentence  highlight:10991  abstract:25215


In [3]:
from configparser import ConfigParser, ExtendedInterpolation
from typing import Dict, Any

def load_config(config_file: str) ->dict:
    config = ConfigParser(interpolation=ExtendedInterpolation())
    config.optionxform = lambda option: option
    config.read(config_file)

    config_dct: Dict[str, Dict] = dict()
    for section in config.sections():
        tmp_dct: Dict[str, Any] = dict()

        for key, value in config.items(section):
            if value == '':
                tmp_dct[key] = None
                continue
            try:
                tmp_dct[key] = eval(value)
            except NameError:
                print("Note the configuration file format")
        
        config_dct[section] = tmp_dct

    return config_dct

config = load_config('prompt.ini')

prefix = config['prompt']['template_1'] + config['prompt']['label_desc']

In [26]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = 'sk-lvLL8tKYmNPZPJaJ00Ce9b9165A9482f9b43625c3a9cC67a',
    base_url = 'https://api.gpt.ge/v1'
)

In [5]:
import pandas as pd
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

jis_example = pd.read_excel('SentenceLabel/label_example.xlsx')
texts = jis_example['Text'].tolist()
labels = jis_example['Label'].tolist()

examples = []

for i, text in enumerate(texts):
    example = {}
    example['input'] = text
    example['output'] = labels[i]
    examples.append(example)

example_formatter_template = """
input:{input}
output:{output}
"""

example_prompt = PromptTemplate(
    input_variables= ['input','output'],
    template = example_formatter_template
)

suffix = '\ninput:'

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables = ['input'],
    example_separator = '\n',
)

example_strings = [few_shot_prompt.example_prompt.format(**example) for example in examples]
pieces = [prefix, *example_strings, suffix]
template = ''.join([piece for piece in pieces if piece])
# print(template)

In [6]:
model_name = 'gpt-3.5-turbo-0125'
system_message = template

In [19]:
pred_labels = []

In [28]:
len(pred_labels)

13291

In [29]:
from tqdm import tqdm                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

start_pos = 11917+len(pred_labels)
for text in tqdm(at_sens[start_pos:]):
    completion = client.chat.completions.create(
        model = model_name,
        messages=[
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': text+'\noutput:'}
        ]
    )
    label = completion.choices[0].message.content
    pred_labels.append(label)

100%|██████████| 7/7 [00:10<00:00,  1.53s/it]


In [30]:
df = pd.DataFrame()
df['text'] = at_sens[11917:11917+len(pred_labels)]
df['label'] = pred_labels
df.to_excel('SentenceLabel/LIS_A_Pred_Other.xlsx', index=False)

##### Filter

In [18]:
# 标签过滤
import pandas as pd
df = pd.read_excel('SentenceLabel/LIS_A_Pred.xlsx')
labels = df['label'].tolist()
texts = df['text'].tolist()

standard_labels = ['class_1: Research and introduction',
                   'class_2: Purpose and background',
                   'class_3: Process and method',
                   'class_4: Result or Conclusion',
                   'class_5: Contribution and promotion']

new_labels = []
new_links = []
new_texts = []
for i, label in enumerate(labels):
    cnt = 0
    new_label = ''
    if type(label) == str:
        for standard_label in standard_labels:
            filter_standard_label = standard_label.lower().replace('_','').replace(' ','').replace('and','')
            filter_label = label.lower().strip().replace('_','').replace(' ','').replace('and','')
            if filter_standard_label in filter_label:
                new_label = standard_label
                cnt += 1

    if cnt == 1:
        new_labels.append(new_label)
        new_links.append(ht_links[i])
        new_texts.append(texts[i])
    # else:
    #     new_labels.append(label)

print(len(df))
print(len(new_links))

10991
10849


In [19]:
df = pd.DataFrame()
df['label'] = new_labels
df['link'] = new_links
df['text'] = new_texts
df = df.dropna().reset_index(drop=True)
df.to_excel('SentenceLabel/LIS_A_Pred_Standard.xlsx', index=False)